# A TensorFlow Convolutional Neural Network on CIFAR-10

In [119]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt 
%matplotlib inline

import keras
from keras.datasets import cifar10

In [96]:
batch_size = 32
num_classes = 10
epochs = 1 

img_row = 32
img_col = 32

In [117]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [120]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("y_train shape:", y_train.shape)

y_train shape: (50000, 10)


In [121]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train.shape

(50000, 32, 32, 3)

In [17]:
x = tf.placeholder(tf.float32, shape=[None, img_row, img_col, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

keep_prob = tf.placeholder(tf.float32)

In [107]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [43]:
with tf.variable_scope("conv_block1") as scope:
    W1_conv1 = weight_variable([3, 3, 3, 32])
    b1_conv1 = bias_variable([32])
    
    conv1_weight1 = tf.nn.conv2d(x, W1_conv1, strides=[1, 1, 1, 1], padding='SAME')
    conv1_bias1 = tf.nn.bias_add(conv1_weight1, b1_conv1)
    conv1_act1 = tf.nn.relu(conv1_bias1, name='conv1_act1')
    
    W2_conv1 = weight_variable([3, 3, 32, 32])
    b2_conv1 = bias_variable([32])
    
    conv1_weight2 = tf.nn.conv2d(conv1_act1, W2_conv1, strides=[1, 1, 1, 1], padding='VALID')
    conv1_bias2 = tf.nn.bias_add(conv1_weight2, b2_conv1)
    conv1_act2 = tf.nn.relu(conv1_bias2, name='conv1_act2')
    
    pool1 = tf.nn.max_pool(conv1_act2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    conv1_dropout = tf.nn.dropout(pool1, keep_prob=0.25, name='dropout1')
    print(conv1_dropout.shape)

(?, 15, 15, 32)


In [44]:
with tf.variable_scope("conv_block2") as scope:
    W1_conv2 = weight_variable([3, 3, 32, 64])
    b1_conv2 = bias_variable([64])
    
    conv2_weight1 = tf.nn.conv2d(conv1_dropout, W1_conv2, strides=[1, 1, 1, 1], padding='SAME')
    conv2_bias1 = tf.nn.bias_add(conv2_weight1, b1_conv2)
    conv2_act1 = tf.nn.relu(conv2_bias1, name='conv2_act1')
    
    W2_conv2 = weight_variable([3, 3, 64, 64])
    b2_conv2 = bias_variable([64])
    
    conv2_weight2 = tf.nn.conv2d(conv2_act1, W2_conv2, strides=[1, 1, 1, 1], padding='VALID')
    conv2_bias2 = tf.nn.bias_add(conv2_weight2, b2_conv2)
    conv2_act2 = tf.nn.relu(conv2_bias2, name='conv2_act2')
    
    pool2 = tf.nn.max_pool(conv2_act2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
    conv2_dropout = tf.nn.dropout(pool2, keep_prob=0.25, name='dropout2')
    print(conv2_dropout.shape)

(?, 6, 6, 64)


In [110]:
with tf.variable_scope("fully-connected3") as scope:
    dense_dim = tf.cast(conv2_dropout.shape[1] * conv2_dropout.shape[2] * conv2_dropout.shape[3], tf.int32)
    # (32, 6, 6, 64) - > (32, 2304)
    # dense_dim = 6*6*64
    dense = tf.reshape(conv2_dropout, [batch_size, -1])   
    
    W_fc3 = weight_variable([dense_dim, 512])
    b_fc3 = bias_variable([512])
    
    fc3_act = tf.nn.relu(tf.matmul(dense, W_fc3) + b_fc3, name='fully-connected3')
    fc3_dropout = tf.nn.dropout(fc3_act, keep_prob=0.5, name='dropout3')

In [111]:
with tf.variable_scope("output") as scope:
    W_fc4 = weight_variable([512, 10])
    b_fc4 = bias_variable([10])
    
    y_pred = tf.nn.softmax(tf.matmul(fc3_dropout, W_fc4) + b_fc4, name='otuput')
    print(y_pred.shape)

(32, 10)


In [112]:
xentropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_))

In [113]:
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(xentropy_loss)

In [114]:
correct_predictions = tf.equal(tf.argmax(y_, 1), tf.argmax(y_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [115]:
init_op = tf.global_variables_initializer()

## Start training

In [ ]:
sess = tf.Session()
sess.run(init_op)

start_time = time.time()

for i in range(epoch):
    # get the next batch of samples with batch size 50
    batch_train = .next_batch(50)   
    # evaluate the loss, run optimizer to train model parameters 
    loss, _  = sess.run([entropy_loss, optimizer], feed_dict={x: batch_train[0], y_: batch_train[1], keep_prob: 0.5})
    
    if i % 500 == 0:
        # evaluate training accuracy, dropout rate set to 1.0 to disable dropout layers 
        acc = sess.run([accuracy], feed_dict={x: batch_train[0], y_: batch_train[1], keep_prob: 1.0})[0]
        # evaluate test accuracy, use test set for feed_dict
        test_accuracy = sess.run([accuracy], feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})[0]
        
        # calculate elapsed time 
        elapsed_time = time.time() - start_time
        start_time = time.time()
        
        print("At step: %d (current time: %g), loss: %g, training accuracy: %g" % (i, elapsed_time, loss, acc))
        print("test accuracy %g\n" % test_accuracy)
